## Task 1. Implement Wagner-Fischer (or Vintsyuk algorithm) 

In [13]:
import numpy as np
import nltk
from nltk.corpus import gutenberg
from collections import Counter

In [24]:

qwerty_keyboard = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

def qwerty_distance(char1, char2):
    if char1 == char2:
        return 0
    elif char1 not in qwerty_keyboard or char2 not in qwerty_keyboard:
        return 1
    else:
        pos1 = qwerty_keyboard[char1]
        pos2 = qwerty_keyboard[char2]
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def wagner_fischer_distance(s1, s2):
    n, m = len(s1), len(s2)
    distance_matrix = np.zeros((n+1, m+1), dtype=int)
    
    for i in range(1, n+1):
        distance_matrix[i][0] = i
    for j in range(1, m+1):
        distance_matrix[0][j] = j
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = qwerty_distance(s1[i-1], s2[j-1])
            distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1, 
                                        distance_matrix[i][j-1] + 1,
                                        distance_matrix[i-1][j-1] + cost) 
            
            if i > 1 and j > 1 and s1[i-1] == s2[j-2] and s1[i-2] == s2[j-1]:
                distance_matrix[i][j] = min(distance_matrix[i][j], distance_matrix[i-2][j-2] + cost) # транспозиція

    return distance_matrix[n][m]

# Приклад використання:
s1 = "meow"
s2 = "dreams"
distance = wagner_fischer_distance(s1, s2)
print("Відстань Вагнера-Фішера між '{}' та '{}' дорівнює: {}".format(s1, s2, distance))


Відстань Вагнера-Фішера між 'meow' та 'dreams' дорівнює: 7


In [32]:
import numpy as np

# Визначаємо розташування клавіш на QWERTY-клавіатурі
qwerty_keyboard = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

def qwerty_distance(char1, char2):
    if char1 == char2:
        return 0
    elif char1 not in qwerty_keyboard or char2 not in qwerty_keyboard:
        return 1
    else:
        pos1 = qwerty_keyboard[char1]
        pos2 = qwerty_keyboard[char2]
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def damerau_levenshtein_distance(s1, s2):
    n, m = len(s1), len(s2)
    distance_matrix = np.zeros((n+1, m+1), dtype=int)
    
    for i in range(1, n+1):
        distance_matrix[i][0] = i
    for j in range(1, m+1):
        distance_matrix[0][j] = j
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if s1[i-1] == s2[j-1]:
                cost = 0
            else:
                cost = qwerty_distance(s1[i-1], s2[j-1])
            distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1,  # видалення
                                        distance_matrix[i][j-1] + 1,  # вставка
                                        distance_matrix[i-1][j-1] + cost)  # заміна
            
            if i > 1 and j > 1 and s1[i-1] == s2[j-2] and s1[i-2] == s2[j-1]:
                distance_matrix[i][j] = min(distance_matrix[i][j], distance_matrix[i-2][j-2] + cost)  # транспозиція

    return distance_matrix[n][m]

# Приклад використання:
s1 = "meow"
s2 = "dreams"
distance = damerau_levenshtein_distance(s1, s2)
print("Відстань Дамерау-Левенштейна між '{}' та '{}' дорівнює: {}".format(s1, s2, distance))


Відстань Дамерау-Левенштейна між 'meow' та 'dreams' дорівнює: 7


## Task 2

In [16]:
my_text = gutenberg.raw('bible-kjv.txt')

# Tokenize the text into sentences
sentences = nltk.sent_tokenize(my_text)

# Tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sentence.lower()) for sentence in sentences]

# Create a vocabulary
vocabulary = set(word for sentence in tokenized_sentences for word in sentence)
word_to_index = {word: i+1 for i, word in enumerate(vocabulary)} # Assign indices to words starting from 1

# Create Bag-of-Words representation for each sentence
bow_tagged_sentences = []
for sentence in tokenized_sentences:
    bow_vector = Counter(sentence)
    bow_tagged_sentence = [(word_to_index[word], count) for word, count in bow_vector.items()]
    bow_tagged_sentences.append(bow_tagged_sentence)

print("Vocabulary:")
print(word_to_index)

print("\nExample of Bag-of-Words representation for the first sentence:")
print(bow_tagged_sentences[0])

Vocabulary:
{'judea': 1, 'winds': 2, 'neglected': 3, 'fainthearted': 4, '7:38': 5, '11:35': 6, 'sychar': 7, 'weaned': 8, 'needeth': 9, 'onan': 10, 'elam': 11, 'leaning': 12, '11:52': 13, 'consisteth': 14, 'migdalel': 15, '62:4': 16, 'proofs': 17, '10:41': 18, '78:32': 19, 'congealed': 20, 'smith': 21, 'baruch': 22, 'anoint': 23, 'azaliah': 24, 'tattlers': 25, 'distil': 26, 'skipping': 27, '146:3': 28, '118:13': 29, 'hanging': 30, 'cedron': 31, 'speed': 32, '13:11': 33, 'hamulites': 34, 'tasteth': 35, 'turning': 36, 'travailed': 37, 'hoofs': 38, 'bringers': 39, 'wolf': 40, 'emmor': 41, 'jahleel': 42, '66:8': 43, 'reasoned': 44, '21:45': 45, 'craftiness': 46, 'fear': 47, '69:8': 48, 'goads': 49, 'peninnah': 50, 'relieveth': 51, 'esek': 52, 'reapers': 53, 'sprigs': 54, 'coupling': 55, '148:13': 56, 'devised': 57, 'dwelled': 58, 'bedeiah': 59, 'baca': 60, 'hinnom': 61, 'colosse': 62, 'urim': 63, '137:1': 64, 'proud': 65, 'unbelieving': 66, 'unseemly': 67, 'balancings': 68, '42:22': 69, 'st

In [33]:
from collections import defaultdict

# Sample document
d = "This is a sample document. We will count the occurrences of each word in this document."

tokenized_document = nltk.word_tokenize(d.lower())

# Count the occurrences of each word in the document
word_counts = defaultdict(int)
for word in tokenized_document:
    if word in word_to_index:
        word_index = word_to_index[word] # Retrieve the index of the word from the vocabulary
        word_counts[word_index] += 1

print("Word counts in document:")
for word_index, count in word_counts.items():
    print(f"({word_index}, d): {count}")


Word counts in document:
(2299, d): 2
(14600, d): 1
(13573, d): 1
(2664, d): 2
(7159, d): 1
(308, d): 1
(14389, d): 1
(4866, d): 1
(12178, d): 1
(16279, d): 1
(4554, d): 1
(12370, d): 1


In [38]:
# Map the document to an N-dimensional vector
N = len(word_to_index)
document_vector = [0] * N

# Update the vector with the counts of each word in the document
for word_index, count in word_counts.items():
    document_vector[word_index - 1] = count  # Adjust index since word indices start from 1

print("Document vector:")
print(document_vector)


Document vector:
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [45]:
import numpy as np

d1 = "This is a sample document."
d2 = "Another document with some different words."

# Tokenize the documents into words
tokenized_d1 = nltk.word_tokenize(d1.lower())
tokenized_d2 = nltk.word_tokenize(d2.lower())

# Count the occurrences of each word in the documents
word_counts_d1 = defaultdict(int)
word_counts_d2 = defaultdict(int)

for word in tokenized_d1:
    if word in word_to_index:
        word_index = word_to_index[word]
        word_counts_d1[word_index] += 1

for word in tokenized_d2:
    if word in word_to_index:
        word_index = word_to_index[word]
        word_counts_d2[word_index] += 1

# Map the documents to N-dimensional vectors
document_vector_d1 = np.array([word_counts_d1.get(i, 0) for i in range(1, N+1)])
document_vector_d2 = np.array([word_counts_d2.get(i, 0) for i in range(1, N+1)])

# Compute the dot product
dot_product = np.dot(document_vector_d1, document_vector_d2)

print("Dot product between the two documents:", dot_product)

Dot product between the two documents: 1
